<a href="https://colab.research.google.com/github/MadimuriVineela/ecommerce/blob/main/cnn%2Brnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip my_data.zip -d /content/my_data

Archive:  my_data.zip
   creating: /content/my_data/my_data/
   creating: /content/my_data/my_data/clean_test/
  inflating: /content/my_data/my_data/clean_test/sp25_clean145.wav  
  inflating: /content/my_data/my_data/clean_test/sp25_clean146.wav  
  inflating: /content/my_data/my_data/clean_test/sp25_clean147.wav  
  inflating: /content/my_data/my_data/clean_test/sp25_clean148.wav  
  inflating: /content/my_data/my_data/clean_test/sp25_clean149.wav  
  inflating: /content/my_data/my_data/clean_test/sp25_clean150.wav  
  inflating: /content/my_data/my_data/clean_test/sp26_clean151.wav  
  inflating: /content/my_data/my_data/clean_test/sp26_clean152.wav  
  inflating: /content/my_data/my_data/clean_test/sp26_clean153.wav  
  inflating: /content/my_data/my_data/clean_test/sp26_clean154.wav  
  inflating: /content/my_data/my_data/clean_test/sp26_clean155.wav  
  inflating: /content/my_data/my_data/clean_test/sp26_clean156.wav  
  inflating: /content/my_data/my_data/clean_test/sp27_clean15

In [ ]:
'''import os

def rename_files(clean_dir, noisy_dir, prefix):
    # List and sort files to maintain order
    clean_files = sorted(os.listdir(clean_dir))
    noisy_files = sorted(os.listdir(noisy_dir))

    # Ensure same number of files
    if len(clean_files) != len(noisy_files):
        raise ValueError(f"Mismatch in number of clean and noisy files in {prefix} dataset!")

    # Rename files sequentially
    for idx, (clean_file, noisy_file) in enumerate(zip(clean_files, noisy_files), start=1):
        clean_ext = clean_file.split(".")[-1]  # Get file extension (e.g., wav)
        noisy_ext = noisy_file.split(".")[-1]

        new_clean_name = f"{prefix}_audio{idx}.{clean_ext}"
        new_noisy_name = f"{prefix}_audio{idx}.{noisy_ext}"

        os.rename(os.path.join(clean_dir, clean_file), os.path.join(clean_dir, new_clean_name))
        os.rename(os.path.join(noisy_dir, noisy_file), os.path.join(noisy_dir, new_noisy_name))

        print(f"Renamed: {clean_file} -> {new_clean_name}, {noisy_file} -> {new_noisy_name}")

# Define dataset paths
datasets = {
    "train": ("/content/my_data/my_data/clean_train", "/content/my_data/my_data/noisy_train"),
    "test": ("/content/my_data/my_data/clean_test", "/content/my_data/my_data/noisy_test")
}

# Apply renaming for both train and test
for prefix, (clean_path, noisy_path) in datasets.items():
    rename_files(clean_path, noisy_path, prefix)
'''

Renamed: sp01_clean1.wav -> train_audio1.wav, sp01_noisy1.wav -> train_audio1.wav
Renamed: sp01_clean2.wav -> train_audio2.wav, sp01_noisy2.wav -> train_audio2.wav
Renamed: sp01_clean3.wav -> train_audio3.wav, sp01_noisy3.wav -> train_audio3.wav
Renamed: sp01_clean4.wav -> train_audio4.wav, sp01_noisy4.wav -> train_audio4.wav
Renamed: sp01_clean5.wav -> train_audio5.wav, sp01_noisy5.wav -> train_audio5.wav
Renamed: sp01_clean6.wav -> train_audio6.wav, sp01_noisy6.wav -> train_audio6.wav
Renamed: sp02 _clean10.wav -> train_audio7.wav, sp02_noisy10.wav -> train_audio7.wav
Renamed: sp02 _clean11.wav -> train_audio8.wav, sp02_noisy11.wav -> train_audio8.wav
Renamed: sp02 _clean12.wav -> train_audio9.wav, sp02_noisy12.wav -> train_audio9.wav
Renamed: sp02 _clean7.wav -> train_audio10.wav, sp02_noisy7.wav -> train_audio10.wav
Renamed: sp02 _clean8.wav -> train_audio11.wav, sp02_noisy8.wav -> train_audio11.wav
Renamed: sp02 _clean9.wav -> train_audio12.wav, sp02_noisy9.wav -> train_audio12.wa

In [ ]:
!pip install torchaudio librosa matplotlib

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from scipy.io import wavfile

In [ ]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

class AudioDataset(Dataset):
    def __init__(self, clean_dir, noisy_dir, chunk_size=16384):
        self.clean_dir = clean_dir
        self.noisy_dir = noisy_dir
        self.chunk_size = chunk_size
        self.file_list = os.listdir(clean_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        clean_path = os.path.join(self.clean_dir, self.file_list[idx])
        noisy_path = os.path.join(self.noisy_dir, self.file_list[idx])

        # Load audio
        clean_waveform, _ = torchaudio.load(clean_path)
        noisy_waveform, _ = torchaudio.load(noisy_path)

        # Convert to mono
        clean_waveform = clean_waveform.mean(dim=0, keepdim=True)
        noisy_waveform = noisy_waveform.mean(dim=0, keepdim=True)

        # Random chunking
        if clean_waveform.shape[1] >= self.chunk_size:
            start = torch.randint(0, clean_waveform.shape[1] - self.chunk_size, (1,))
            clean_waveform = clean_waveform[:, start:start+self.chunk_size]
            noisy_waveform = noisy_waveform[:, start:start+self.chunk_size]
        else:
            # Pad if too short
            padding = self.chunk_size - clean_waveform.shape[1]
            clean_waveform = torch.nn.functional.pad(clean_waveform, (0, padding))
            noisy_waveform = torch.nn.functional.pad(noisy_waveform, (0, padding))

        return noisy_waveform, clean_waveform

In [ ]:
# Update these paths according to your dataset location
train_clean = "/content/my_data/my_data/clean_train"
train_noisy = "/content/my_data/my_data/noisy_train"
test_clean = "/content/my_data/my_data/clean_test"
test_noisy = "/content/my_data/my_data/noisy_test"

In [ ]:
train_dataset = AudioDataset(train_clean, train_noisy)
test_dataset = AudioDataset(test_clean, test_noisy)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_RNN_GRU(nn.Module):
    def __init__(self):
        super(CNN_RNN_GRU, self).__init__()

        # CNN for feature extraction
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=15, padding=7),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.2),
            nn.Conv1d(32, 64, kernel_size=15, padding=7),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Conv1d(64, 128, kernel_size=15, padding=7),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2)
        )

        # RNN Layer
        self.rnn = nn.RNN(input_size=128, hidden_size=128, num_layers=1, batch_first=True)

        # GRU Layer
        self.gru = nn.GRU(input_size=128, hidden_size=128, num_layers=2, batch_first=True, bidirectional=True)

        # Fully Connected Layer
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        # CNN feature extraction
        x = self.cnn(x)

        # Reshape for RNN
        x = x.permute(0, 2, 1)  # [batch_size, length, channels]

        # RNN Layer
        x, _ = self.rnn(x)  # Simple RNN processing

        # GRU Layer
        x, _ = self.gru(x)  # GRU processing

        # Fully connected layers
        x = self.fc(x)  # [batch_size, length, 1]
        x = x.permute(0, 2, 1)  # [batch_size, 1, length]

        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_RNN_GRU().to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

In [ ]:
import torch.optim as optim
from tqdm import tqdm

def train(model, dataloader, epoch, device, optimizer, criterion):
    model.train()
    total_loss = 0
    progress = tqdm(dataloader, desc=f"Epoch {epoch+1}")

    for noisy, clean in progress:
        noisy = noisy.to(device)
        clean = clean.to(device)

        optimizer.zero_grad()
        outputs = model(noisy)
        loss = criterion(outputs, clean)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress.set_postfix({'loss': loss.item()})

    return total_loss / len(dataloader)

def validate(model, dataloader, device, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for noisy, clean in dataloader:
            noisy = noisy.to(device)
            clean = clean.to(device)

            outputs = model(noisy)
            loss = criterion(outputs, clean)
            total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
best_val_loss = float('inf')
train_losses = []
val_losses = []

num_epochs = 20
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, epoch, device, optimizer, criterion)
    val_loss = validate(model, test_loader, device, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Model saved!")

    scheduler.step(val_loss)

Epoch 1: 100%|██████████| 18/18 [16:24<00:00, 54.72s/it, loss=0.0193]


Epoch 1/20
Train Loss: 0.0464 | Val Loss: 0.0224
Model saved!


Epoch 2:  11%|█         | 2/18 [01:46<14:00, 52.50s/it, loss=0.0167]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Training Progress")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()